In [1]:
import os
import sys
import pymysql

In [2]:
%pwd

'e:\\ML_Projects\\Predictive_Maintenance\\notebooks'

In [3]:
os.chdir("../")

In [4]:
%pwd

'e:\\ML_Projects\\Predictive_Maintenance'

In [5]:
from src.exception import CustomException
from src.logger import logging
import pandas as pd 
from dataclasses import dataclass

In [6]:
@dataclass
class DataIngestionConfig:
    train_data_path:str = os.path.join("artifacts", "train.csv")
    test_data_path:str = os.path.join("artifacts", "test.csv")

In [7]:
from dotenv import load_dotenv

host = os.getenv("host")
user = os.getenv("user")
password = os.getenv("password")
db = os.getenv("db")

In [8]:
def read_sql_data():
    logging.info("Reading SQL Database")
    
    try:
        mydb = pymysql.connect(
            host = host,
            user = user,
            password = password,
            db = db
        )
        logging.info("Connection establish")
        
        df_train = pd.read_sql_query("select * from train1", mydb)
        logging.info("Train readed")
        df_test = pd.read_sql_query("select * from test1", mydb)
        logging.info("Test readed")
        
        print(df_train.head())
        print(df_test.head())
        
        return df_train, df_test
        
    except Exception as e:
        raise CustomException(e, sys)

In [9]:
class DataIngestion:
    
    def __init__(self):
        self.ingestion_config = DataIngestionConfig()
        
    def initiate_data_ingestion(self):
        try:
            df = read_sql_data()
            logging.info("Reading from SQL database")
            
            os.makedirs(os.path.dirname(self.ingestion_config.train_data_path), exist_ok = True)
            os.makedirs(os.path.dirname(self.ingestion_config.test_data_path), exist_ok = True)
            
            df[0].to_csv(self.ingestion_config.train_data_path, index = False, header = True)
            df[1].to_csv(self.ingestion_config.test_data_path, index = False, header = True)
            
            logging.info("Data Ingestion Completed")
            
            return(
                self.ingestion_config.train_data_path,
                self.ingestion_config.test_data_path
            )
        except Exception as e:
            raise CustomException(e, sys)

In [10]:
try:
    # data_ingestion_config = DataIngestionConfig()
    data_ingestion = DataIngestion()
    data_ingestion.initiate_data_ingestion()
    
except Exception as e:
    logging.info("Custom Exception")
    raise CustomException(e, sys)

C:\Users\Ayush Gandhi\AppData\Local\Temp\ipykernel_16196\1192899679.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_train = pd.read_sql_query("select * from train1", mydb)
C:\Users\Ayush Gandhi\AppData\Local\Temp\ipykernel_16196\1192899679.py:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_test = pd.read_sql_query("select * from test1", mydb)


   Engine_no  Cycle_no  Setting_1  Setting_2  Setting_3  \
0          1         1    -0.0007    -0.0004      100.0   
1          1         2     0.0019    -0.0003      100.0   
2          1         3    -0.0043     0.0003      100.0   
3          1         4     0.0007     0.0000      100.0   
4          1         5    -0.0019    -0.0002      100.0   

   Fan_inlet_temperature  LPC_outlet_temperature  HPC_outlet_temperature  \
0                 518.67                  641.82                 1589.70   
1                 518.67                  642.15                 1591.82   
2                 518.67                  642.35                 1587.99   
3                 518.67                  642.35                 1582.79   
4                 518.67                  642.37                 1582.85   

   LPT_outlet_temperature  Fan_inlet_pressure  ...  Fan_speed  Core_speed  \
0                 1400.60               14.62  ...    2388.02     8138.62   
1                 1403.14         